In [1]:
import numpy as np

import ufl
import basix
from dolfinx import mesh, fem, io
from ufl import ds, dx

from mpi4py import MPI
from petsc4py import PETSc

In [2]:
import dolfinx
dolfinx.__version__

'0.3.1.0'

In [3]:
def project(original_field, target_field, dx=ufl.dx, bcs=[]):
    # original_field -> target_field
    # Ensure we have a mesh and attach to measure
    V = target_field.function_space

    # Define variational problem for projection
    w = ufl.TestFunction(V)
    Pv = ufl.TrialFunction(V)
    a = fem.form(ufl.inner(Pv, w) * dx)
    L = fem.form(ufl.inner(original_field, w) * dx)

    # Assemble linear system
    A = fem.petsc.assemble_matrix(a, bcs)
    A.assemble()
    b = fem.petsc.assemble_vector(L)
    fem.petsc.apply_lifting(b, [a], [bcs])
    b.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)
    fem.petsc.set_bc(b, bcs)

    # Solve linear system
    solver = PETSc.KSP().create(A.getComm())
    solver.setOperators(A)
    solver.solve(b, target_field.vector)  
    target_field.x.scatter_forward()

In [4]:
L = 1
W = 0.1
mu = 1
rho = 1
delta = W/L
gamma = 0.4*delta**2
beta = 1.25
lambda_ = beta
g = gamma

In [5]:
# Create mesh and define function space
domain = mesh.create_rectangle(comm=MPI.COMM_WORLD,
                            points=((0.0, 0.0), (L, W)), n=(64, 16),
                            cell_type=mesh.CellType.triangle,)

dxm = ufl.Measure(
    "dx",
    domain=domain,
    metadata={"quadrature_degree": 2, "quadrature_scheme": "default"},
)

deg_u = 1
deg_stress = 0
deg_q = 1

DIM = domain.geometry.dim # == domain.topology.dim

QV0e = ufl.VectorElement("Quadrature", domain.ufl_cell(), degree=1, dim=3, quad_scheme='default')
QV0dim2e = ufl.VectorElement("Quadrature", domain.ufl_cell(), degree=1, dim=2, quad_scheme='default')
DGV0e = ufl.VectorElement("DG", domain.ufl_cell(), degree=0, dim=3)
QV2e = ufl.VectorElement("Quadrature", domain.ufl_cell(), degree=2, dim=3, quad_scheme='default')

Q0 = fem.FunctionSpace(domain, ufl.FiniteElement("Quadrature", domain.ufl_cell(), degree=0, quad_scheme='default'))
Q1 = fem.FunctionSpace(domain, ufl.FiniteElement("Quadrature", domain.ufl_cell(), degree=1, quad_scheme='default'))
Q2 = fem.FunctionSpace(domain, ufl.FiniteElement("Quadrature", domain.ufl_cell(), degree=2, quad_scheme='default'))
QV0 = fem.FunctionSpace(domain, QV0e)
QV0dim2 = fem.FunctionSpace(domain, QV0dim2e)
QV2 = fem.FunctionSpace(domain, QV2e)

DG0 = fem.FunctionSpace(domain, ('DG', 0))
DG1 = fem.FunctionSpace(domain, ('DG', 1))
DGV0 = fem.FunctionSpace(domain, DGV0e)

CG1 = fem.FunctionSpace(domain, ('CG', 1))
V = fem.VectorFunctionSpace(domain, ("Lagrange", deg_u))
#Q0 == Q1 == DG0

In [29]:
num_nodes_global = domain.topology.index_map(domain.topology.dim-2).size_global
num_cells_global = domain.topology.index_map(domain.topology.dim).size_global

# num_dofs_local = (V.dofmap.index_map.size_local) #* V.dofmap.index_map_bs
num_dofs_global = V.dofmap.index_map.size_global #* V.dofmap.index_map_bs
# num_dofs = domain.topology.index_map(domain.topology.dim).size_local 
# V.num_sub_spaces == V.dofmap.index_map_bs
# print(f"Number of dofs (owned) by rank : {num_dofs_local}")
if MPI.COMM_WORLD.rank == 0:
    print(f"Nodes global = {num_nodes_global}, Cells global = {num_cells_global}")
    print(f"Number of dofs global V: {num_dofs_global}")
    print(f"Number of dofs global Q0: {Q0.dofmap.index_map.size_global}")
    print(f"Number of dofs global Q1: {Q1.dofmap.index_map.size_global}")
    print(f"Number of dofs global Q2: {Q2.dofmap.index_map.size_global}")
    print(f"Number of dofs global DG0: {DG0.dofmap.index_map.size_global}")
    print(f"Number of dofs global DG1: {DG1.dofmap.index_map.size_global}")
    print(f"Number of dofs global CG1: {CG1.dofmap.index_map.size_global}")

Nodes global = 1105, Cells global = 2048
Number of dofs global V: 1105
Number of dofs global Q0: 2048
Number of dofs global Q1: 2048
Number of dofs global Q2: 6144
Number of dofs global DG0: 2048
Number of dofs global DG1: 6144
Number of dofs global CG1: 1105


In [30]:
facets = mesh.locate_entities_boundary(domain, dim=1,
                                       marker=lambda x: np.isclose(x[0], 0.0))

dofs = fem.locate_dofs_topological(V=V, entity_dim=1, entities=facets)
bc = fem.dirichletbc(value=fem.Constant(domain, (PETSc.ScalarType(0), PETSc.ScalarType(0))), dofs=dofs, V=V)

In [31]:
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

def epsilon(u):
    return ufl.sym(ufl.grad(u))

def sigma(u):
    return lambda_*ufl.div(u)*ufl.Identity(u.geometric_dimension()) + 2*mu*epsilon(u)

# Define variational problem
f = fem.Constant(domain, (PETSc.ScalarType(0), PETSc.ScalarType(-rho*g)))
T = fem.Constant(domain, (PETSc.ScalarType(0), PETSc.ScalarType(0)))
a = ufl.inner(sigma(u), epsilon(v))*dx
b = ufl.inner(f, v)*dx + ufl.inner(T, v)*ds

problem = fem.petsc.LinearProblem(a, b, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()
uh.x.scatter_forward()

A = fem.petsc.create_matrix(fem.form(a))
B = fem.petsc.create_vector(fem.form(b))
uh2 = fem.Function(V)

A.zeroEntries()
fem.petsc.assemble_matrix(A, fem.form(a), bcs=[bc])
A.assemble()

with B.localForm() as B_local:
    B_local.set(0.0)
fem.petsc.assemble_vector(B, fem.form(b))

fem.apply_lifting(B, [fem.form(a)], bcs=[[bc]])
B.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)
fem.set_bc(B, [bc])

solver = PETSc.KSP().create(A.getComm())
solver.setOperators(A)
solver.solve(B, uh2.vector)
solver.setType(PETSc.KSP.Type.PREONLY)
solver.getPC().setType(PETSc.PC.Type.LU)
uh2.x.scatter_forward()

In [32]:
# defining function to interpolate function defined over quadrature elements
def interpolate_quadrature(ufl_expr, fem_func):
    q_dim = fem_func.function_space._ufl_element.degree()
    mesh = fem_func.ufl_function_space().mesh
    
    basix_celltype = getattr(basix.CellType, domain.topology.cell_type.name)
    quadrature_points, weights = basix.make_quadrature(basix_celltype, q_dim)
    map_c = mesh.topology.index_map(mesh.topology.dim)
    num_cells = map_c.size_local + map_c.num_ghosts
    cells = np.arange(0, num_cells, dtype=np.int32)

    expr_expr = fem.Expression(ufl_expr, quadrature_points)
    expr_eval = expr_expr.eval(cells)
    fem_func.x.array[:] = expr_eval.flatten()[:]
    fem_func.x.scatter_forward()

    # with funct.vector.localForm() as funct_local:
    #     funct_local.setBlockSize(funct.function_space.dofmap.bs)
    #     funct_local.setValuesBlocked(
    #         funct.function_space.dofmap.list.array,
    #         expr_eval,
    #         addv=PETSc.InsertMode.INSERT,
    #     )

In [33]:
sigmah = sigma(uh)
sigma_xx_ = sigmah[0,0]
sigma_xy_ = sigmah[0,1]
sigma_yy_ = sigmah[1,1]

sigma_vec = ufl.as_vector([sigmah[0, 0], sigmah[0, 1], sigmah[1, 1]])
sigma_vec2 = ufl.as_vector([sigmah[0, 0], sigmah[0, 1]])

In [34]:
sigma_xx_Q0 = fem.Function(Q0)
sigma_xx_DG0 = fem.Function(DG0)
sigma_xx_DG1 = fem.Function(DG1)

sigma_xx_DG0_interp = fem.Function(DG0)
sigma_xy_DG0_interp = fem.Function(DG0)
sigma_yy_DG0_interp = fem.Function(DG0)
sigma_DGV0_interp = fem.Function(DGV0)

sigma_xx_DG1_interp = fem.Function(DG1)

sigma_xx_Q0_interp = fem.Function(Q0)
sigma_xy_Q0_interp = fem.Function(Q0)
sigma_yy_Q0_interp = fem.Function(Q0)
sigma_QV0 = fem.Function(QV0)
sigma_QV0dim2 = fem.Function(QV0dim2)

sigma_xx_Q2_interp = fem.Function(Q2)
sigma_xy_Q2_interp = fem.Function(Q2)
sigma_yy_Q2_interp = fem.Function(Q2)
sigma_QV2 = fem.Function(QV2)

sigma_xx_Q2_CG1 = fem.Function(CG1)
sigma_xx_Q2_DG1 = fem.Function(DG1)

In [35]:
project(sigma_xx_, sigma_xx_Q0)
project(sigma_xx_, sigma_xx_DG0)
project(sigma_xx_, sigma_xx_DG1)
# project(sigma_vec, sigma_QV0) #failed

In [36]:
expr = fem.Expression(sigma_xx_, DG0.element.interpolation_points)
sigma_xx_DG0_interp.interpolate(expr)
expr = fem.Expression(sigma_xy_, DG0.element.interpolation_points)
sigma_xy_DG0_interp.interpolate(expr)
expr = fem.Expression(sigma_yy_, DG0.element.interpolation_points)
sigma_yy_DG0_interp.interpolate(expr)

expr = fem.Expression(sigma_xx_, DG1.element.interpolation_points)
sigma_xx_DG1_interp.interpolate(expr)
sigma_xx_DG1_interp.x.scatter_forward()

expr = fem.Expression(sigma_vec, DGV0.element.interpolation_points)
sigma_DGV0_interp.interpolate(expr)

interpolate_quadrature(sigma_xx_, sigma_xx_Q0_interp)
interpolate_quadrature(sigma_vec, sigma_QV0)
interpolate_quadrature(sigma_vec2, sigma_QV0dim2)
interpolate_quadrature(sigma_vec, sigma_QV2)

interpolate_quadrature(sigma_xx_, sigma_xx_Q2_interp)

In [37]:
project(sigma_xx_Q2_interp, sigma_xx_Q2_CG1, dx=dxm)
project(sigma_xx_Q2_interp, sigma_xx_Q2_DG1, dx=dxm)

In [38]:
sigma_xx_Q0_interp, sigma_xy_Q0_interp, sigma_yy_Q0_interp = sigma_QV0.split()
sigma_xx_DG0_interp, sigma_xy_DG0_interp, sigma_yy_DG0_interp = sigma_DGV0_interp.split()
sigma_xx_Q2_interp, sigma_xy_Q2_interp, sigma_yy_Q2_interp = sigma_QV2.split()

In [39]:
print(sigma_QV0.x.array.shape, sigma_xx_Q0_interp.x.array.shape)
print(sigma_DGV0_interp.x.array.shape, sigma_xx_DG0_interp.x.array.shape)

(6144,) (6144,)
(6144,) (6144,)


In [40]:
u = fem.Function(V)
ux, uy = u.split()
print(uy.x.array.shape, ux.x.array.shape, u.sub(0).x.array.shape, u.x.array.shape)

(2210,) (2210,) (2210,) (2210,)


In [41]:
DG0.element.interpolation_points

array([[0.33333333, 0.33333333]])

In [42]:
DG1.element.interpolation_points

array([[0., 0.],
       [1., 0.],
       [0., 1.]])

In [43]:
quadrature_points, _ = basix.make_quadrature(basix.CellType.triangle, 2)
quadrature_points

array([[0.16666667, 0.16666667],
       [0.16666667, 0.66666667],
       [0.66666667, 0.16666667]])

In [44]:
num_ghosts = domain.topology.index_map(domain.topology.dim).num_ghosts
num_cells_local = domain.topology.index_map(domain.topology.dim).size_local #== domain.geometry.dofmap.num_nodes
num_nodes_local = domain.topology.index_map(domain.topology.dim-2).size_local #== domain.geometry.x.shape[0]

print(f"rank = {MPI.COMM_WORLD.rank}:")
print(f"\tNodes local = {num_nodes_local}, Cells local = {num_cells_local}, Cells ghost = {num_ghosts}")
print(f"\tDoFs local = {V.dofmap.index_map.size_local}, Number of uh_x = {uh.x.array.shape[0]/V.num_sub_spaces}, Number of sigma_xx_DG0 = {sigma_xx_DG0.x.array.shape[0]}")

rank = 0:
	Nodes local = 1105, Cells local = 2048, Cells ghost = 0
	DoFs local = 1105, Number of uh_x = 1105.0, Number of sigma_xx_DG0 = 2048


In [45]:
uh.name = "Displacement"
sigma_xx_Q0.name = 'sigma xx Q0'
sigma_xx_DG0.name = 'sigma xx DG0'
sigma_xx_DG1.name = 'sigma xx DG1'
sigma_xx_DG0_interp.name = 'sigma xx DG0 interp'
sigma_xy_DG0_interp.name = 'sigma xy DG0 interp'
sigma_yy_DG0_interp.name = 'sigma yy DG0 interp'

sigma_xx_DG1_interp.name = 'sigma xx DG1 interp'

sigma_xx_Q0_interp.name = 'sigma xx Q0 interp'
sigma_xy_Q0_interp.name = 'sigma xy Q0 interp'
sigma_yy_Q0_interp.name = 'sigma yy Q0 interp'

sigma_xx_Q2_interp.name = 'sigma xx Q2 interp'
sigma_xy_Q2_interp.name = 'sigma xy Q2 interp'
sigma_yy_Q2_interp.name = 'sigma yy Q2 interp'

sigma_xx_Q2_CG1.name = 'sigma xx Q2->CG1'
sigma_xx_Q2_DG1.name = 'sigma xx Q2->DG1'

sigma_DGV0_interp.name = 'sigma DGV0 interp'
sigma_QV0dim2.name = 'sigma QV0dim2'
sigma_QV0.name = "Stress"

with io.XDMFFile(MPI.COMM_WORLD, "solution_0.xdmf", "w", encoding=io.XDMFFile.Encoding.HDF5) as file:
    file.write_mesh(domain)

with io.XDMFFile(MPI.COMM_WORLD, "solution_0.xdmf", "a", encoding=io.XDMFFile.Encoding.HDF5) as file:
    file.write_function(uh)
    file.write_function(uh2)
    file.write_function(sigma_xx_Q0)
    file.write_function(sigma_xx_DG0)
    file.write_function(sigma_xx_DG1)
    file.write_function(sigma_xx_DG0_interp)
    file.write_function(sigma_xy_DG0_interp)
    file.write_function(sigma_yy_DG0_interp)
    file.write_function(sigma_xx_DG1_interp)
    file.write_function(sigma_xx_Q0_interp)
    file.write_function(sigma_xy_Q0_interp)
    file.write_function(sigma_yy_Q0_interp)
    file.write_function(sigma_xx_Q2_CG1)
    file.write_function(sigma_xx_Q2_DG1)
    #TODO: Check points
    # file.write_function(sigma_xx_Q2_interp) #failed!
    file.write_function(sigma_DGV0_interp)
    file.write_function(sigma_QV0dim2)
    # file.write_function(sigma_QV0, 0) #failed!

In [22]:
sigma_xx_DG0.x.array.shape

(2048,)

In [23]:
sigma_xx_DG1.x.array.shape

(6144,)

In [24]:
sigma_DGV0_interp.x.array.shape

(6144,)

In [25]:
sigma_xx_DG0_interp.x.array.shape

(6144,)

In [26]:
# import pyvista
# from dolfinx.plot import create_vtk_mesh

# pyvista.set_jupyter_backend("pythreejs")
# # plotter = pyvista.Plotter()
# grid = pyvista.UnstructuredGrid(*create_vtk_mesh(domain, domain.topology.dim))

# # u_grid = pyvista.UnstructuredGrid(u_topology, u_cell_types, u_geometry)
# # grid.cell_data["sig_xx"] = sigma_xx_DG0.x.array.real
# grid.field_data["sig_xx"] = sigma_xx_DG0.x.array.real
# grid.set_active_scalars("sig_xx")
# u_plotter = pyvista.Plotter()
# u_plotter.add_mesh(grid, show_edges=False)
# u_plotter.view_xy()
# if not pyvista.OFF_SCREEN:
#     u_plotter.show()

# num_local_cells = domain.topology.index_map(domain.topology.dim).size_local
# grid.cell_data["Marker"] = ct.values[ct.indices<num_local_cells]
# grid.set_active_scalars("Marker")
# actor = plotter.add_mesh(grid, show_edges=True)
# plotter.view_xy()
# if not pyvista.OFF_SCREEN:
#     plotter.show()
# else:
#     pyvista.start_xvfb()
#     cell_tag_fig = plotter.screenshot("cell_tags.png")